In [15]:
import os
import json
import warnings
import optuna
import numpy as np
import pandas as pd
from io import StringIO
from pathlib import Path
from datetime import datetime

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor, sum_models

warnings.filterwarnings("ignore")

In [2]:
height = [5, 5, 5, 5, 5, 5, 5, 3, 3, 1, 1, 1, 5, 5, 1, 1, 0]
angle = [181, 175, 180, 161, 208, 208, 172, 219, 151, 223, 131, 298, 249, 197, 127, 82, 0]


for i in range(1, 18):
    dataset_dir = os.getcwd() + f"/36_TrainingData/L{i}_Train.csv"
    print(dataset_dir)
    df = pd.read_csv(dataset_dir)

    # with additional data
    if i in [2, 4, 7, 8, 9, 10, 12]:
        addition_dir = os.getcwd() + f"/36_TrainingData_Additional_V2/L{i}_Train_2.csv"
        df2 = pd.read_csv(addition_dir)
        df = pd.concat([df, df2], axis = False)

    df['DateTime'] = pd.to_datetime(df['DateTime'])
    df['height'] = height[i-1]
    df['angle'] = angle[i-1]
    col = df.pop("Power(mW)")
    df["Power(mW)"] = col

    df.set_index('DateTime', inplace=True)
    df = df.resample('10T').mean()
    df = df.round(2)

    df.reset_index(inplace=True)
    # df.replace("", np.nan, inplace=True)
    df = df.dropna()

    df.to_csv(os.getcwd() + f"/TrainingData_avg/L{i}_Train.csv", index=False)

/home/h24116049/project/AI-CUP-2024/36_TrainingData/L1_Train.csv


/tmp/ipykernel_869503/910421310.py:23: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df = df.resample('10T').mean()


/home/h24116049/project/AI-CUP-2024/36_TrainingData/L2_Train.csv


/tmp/ipykernel_869503/910421310.py:23: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df = df.resample('10T').mean()


/home/h24116049/project/AI-CUP-2024/36_TrainingData/L3_Train.csv


...

/tmp/ipykernel_869503/910421310.py:23: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df = df.resample('10T').mean()


/home/h24116049/project/AI-CUP-2024/36_TrainingData/L16_Train.csv


/tmp/ipykernel_869503/910421310.py:23: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df = df.resample('10T').mean()


/home/h24116049/project/AI-CUP-2024/36_TrainingData/L17_Train.csv


/tmp/ipykernel_869503/910421310.py:23: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df = df.resample('10T').mean()


In [2]:
target_dir = os.getcwd() + f"/upload.csv"
target = pd.read_csv(target_dir, encoding='utf8')['序號'].values

date_device = {}
for item in target:
    date = str(item)[:8]
    device = str(item)[12:]
    dd = date + device

    if dd not in date_device.keys():
        date_device[dd] = [int(device)]
        
data_ls = []

for date in list(date_device.keys()):
    # traverse all dataset to find target days
    for i in range(1, 18):
        data_dir = os.getcwd() + f"/TrainingData_avg/L{i}_Train.csv"
        data = pd.read_csv(data_dir)
        data['DateTime'] = pd.to_datetime(data['DateTime'])
        data_ls.append(data)  # store data

        date_ls = data['DateTime'].dt.strftime("%Y%m%d").values

        if date[:-2] in date_ls and i not in date_device[date]:
            date_device[date].append(i)

# write file        
with open("target_match_device.txt", 'w') as file:
    file.write(json.dumps(date_device))

date_device


In [3]:
def make_date_device(data_ls):
    target_dir = os.getcwd() + f"/sample_upload.csv"
    target = pd.read_csv(target_dir, encoding='utf8')['序號'].values

    date_device = {}
    for item in target:
        date = str(item)[:8]
        device = str(item)[12:]
        dd = date + device

        if dd not in date_device.keys():
            date_device[dd] = [int(device)]
    # data_ls = load_data()

    for date in list(date_device.keys()):
        year = date[:4]
        month = date[4:6]
        day = date[6:-2]

        start_time = pd.Timestamp(f"{year}-{month}-{day} 09:00:00")
        end_time = pd.Timestamp(f"{year}-{month}-{day} 16:50:00")
        # traverse all dataset to find target days
        for i in range(1, 18):
            data = data_ls[i-1]

            filtered_df = data[(data['DateTime'] >= start_time) & (data['DateTime'] <= end_time)]
            if len(filtered_df)>40:
                date_device[date].append(i)
    
    # write file        
    # with open("target_match_device.txt", 'w') as file:
    #     file.write(json.dumps(date_device))

    return date_device

date_device = make_date_device(data_ls)

In [ ]:
# Optuna Objective
def objective(trial):
    param = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 500),
        "max_depth": trial.suggest_int("max_depth", 3, 30),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 20),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 20),
        "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2", None]),
        "bootstrap": trial.suggest_categorical("bootstrap", [True, False]),
    }

    model = RandomForestRegressor(**param, random_state=42, n_jobs=-1)
    model.fit(X_train, y_train)

    
    y_pred = model.predict(X_val)
    mse = mean_squared_error(y_val, y_pred)
    
    return mse

In [9]:
def feature_engineering(df):
    df = df.copy()
    df['hour'] = df['DateTime'].dt.hour
    df['minute'] = df['DateTime'].dt.minute
    baseline_date = pd.to_datetime('2024-01-01')
    df['days'] = (df['DateTime'] - baseline_date).dt.days  # distance to the baseline date(2024-01-01)
    df['month'] = df['DateTime'].dt.month


    # df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
    # df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)

    # df['minute_sin'] = np.sin(2 * np.pi * df['minute'] / 24)
    # df['minute_cos'] = np.cos(2 * np.pi * df['minute'] / 24)

    df['lag_sunlight'] = df['Sunlight(Lux)'].shift(1)
    df['lag_power'] = df['Power(mW)'].shift(1)

    df.drop(columns=['DateTime', 'height', 'angle', 'LocationCode'], inplace=True)
    # df.drop(columns=['DateTime'], inplace=True)
    return df

def load_data():
    data_ls = []
    for i in range(1, 18):
        data_dir = os.getcwd() + f"/TrainingData_avg/L{i}_Train.csv"
        data = pd.read_csv(data_dir)
        data['DateTime'] = pd.to_datetime(data['DateTime'])
        data.dropna(axis=0, how='any', inplace=True)
        data_ls.append(data)  # store data
    
    return data_ls

def load_date_device_dict():
    with open(os.getcwd() + "/target_match_device.txt") as f:
        date_dict = f.read()
    
    return json.loads(date_dict)



def get_intersection_date(data, id1, id2, date_id):
    data1 = data[id1-1]
    data2 = data[id2-1]
    target = data1[data1['DateTime'].isin(data2['DateTime'])].reset_index(drop=True)  # assign divice1 to the target power
    target.rename(columns={'Power(mW)': 'Label'}, inplace=True)
    target = target['Label']
    match2 = data2[data2['DateTime'].isin(data1['DateTime'])].reset_index(drop=True)

    # target test data
    start_time = pd.to_datetime(f"{date_id}0900", format='%Y%m%d%H%M')
    # print(start_time)
    end_time = pd.to_datetime(f"{date_id}1650", format='%Y%m%d%H%M')
    # print(end_time)
    test_data = data2[(data2['DateTime'] >= start_time) & (data2['DateTime'] <= end_time)].reset_index(drop=True)

    match2 = feature_engineering(match2)
    test_data = feature_engineering(test_data)

    return match2, target, test_data


def split_train_val_test(df, label):
    X_train, X_val, y_train, y_val = train_test_split(df, label, test_size=0.2, random_state=42)
    return X_train, X_val, y_train, y_val

def normalize_features(X_train, X_val, X_test):
    min_max_scaler = preprocessing.MinMaxScaler()

    X_train_scaled = min_max_scaler.fit_transform(X_train)
    X_val_scaled = min_max_scaler.transform(X_val)
    X_test_scaled = min_max_scaler.transform(X_test)

    X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns).astype(float)
    X_val_scaled_df = pd.DataFrame(X_val_scaled, columns=X_val.columns).astype(float)
    X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test.columns).astype(float)

    return X_train_scaled_df, X_val_scaled_df, X_test_scaled_df


def complete_target_time():
    target_time = []
    for hour in range(9, 17):
        for minute in range(0, 60, 10):
            if hour == 16 and minute > 50:
                break
            target_time.append((hour, minute))
    
    return target_time


def fillna(X_test):
    target_time = complete_target_time()
    time_df = pd.DataFrame(target_time, columns=["hour", "minute"])

    df = time_df.merge(
        X_test,
        on=["hour", "minute"],
        how="left"
    )
    df.fillna(method="ffill", inplace=True)
    df.fillna(method="bfill", inplace=True) 
    df = df[X_test.columns]

    return df

def get_avg_of_closest4(data, date_id, result):
    df = feature_engineering(data)
    target_time = complete_target_time()

    # days from 2024-01-01
    date = pd.to_datetime(f"{date_id[:4]}-{date_id[4:6]}-{date_id[6:]}")
    baseline_date = pd.to_datetime('2024-01-01')
    target_day = (date - baseline_date).days

    for (hour, minute) in target_time:
        time_data = df[(df['hour'] == hour) & (df['minute'] == minute)]
        date_diff = np.abs(time_data['days'] - target_day)
        closest4_days = time_data.iloc[np.argsort(date_diff)[:4]]
        
        average_power = closest4_days['Power(mW)'].mean()
        result.append(np.round(average_power, 2))
    
    return result


def train_model(X_train, y_train, X_val, y_val, study):

    model = RandomForestRegressor(n_estimators=100, max_depth=6, random_state=42, verbose=0)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    mse = mean_squared_error(y_val, y_pred)
    mae = mean_absolute_error(y_val, y_pred)
    return model, mse, mae

def evaluate_metrics(model, X_val, y_val):
    y_pred = model.predict(X_val)
    mse = mean_squared_error(y_val, y_pred)
    mae = mean_absolute_error(y_val, y_pred)

    return mse, mae

def run():
    data_ls = load_data()
    date_device = load_date_device_dict()
    total_mse = 0
    total_mae = 0
    
    result = []
    model_cache = {}

    # gg = {"2024011701": [1, 8, 17], "2024011901": [1, 2, 8, 17], "2024012502": [2, 1, 8, 17]}

    for idx, (date_id, device_ls) in enumerate(date_device.items()):
        date_id = date_id[:-2]
        best_mse = float('inf')
        best_mae = None
        best_model = None
        best_device = None
        
        sum_results = None

        # no data found in other device, take average of 4 closest days
        if len(device_ls) == 1:
            data = data_ls[device_ls[0]]  # data of target device
            result = get_avg_of_closest4(data, date_id, result)

            print("current problem: {}/200 | No device, fill with average power in 4 days".format(idx+1))

        else:
            model_len = len(device_ls[1:])
            for dev_idx, dev in enumerate(device_ls[1:]):
                train_data, label_data, X_test = get_intersection_date(data_ls, device_ls[0], dev, date_id)
                X_train, X_val, y_train, y_val = split_train_val_test(train_data, label_data)
                if X_test.shape[0] < 48:
                    X_test = fillna(X_test)
                # output[idx].append(48 - X_test.shape[0])

                # X_train, X_val, X_test = normalize_features(X_train, X_val, X_test)
                
                if (device_ls[0], dev) not in list(model_cache.keys()):
                    epoch_model, epoch_mse, epoch_mae = train_model(X_train, y_train, X_val, y_val)
                    model_cache[(device_ls[0], dev)] = [epoch_model, epoch_mse, epoch_mae]  # store pair model in cache
                else:
                    epoch_model, epoch_mse, epoch_mae = model_cache[(device_ls[0], dev)]

                def objective(trial):
                    param = {
                        "n_estimators": trial.suggest_int("n_estimators", 50, 500),
                        "max_depth": trial.suggest_int("max_depth", 3, 30),
                        "min_samples_split": trial.suggest_int("min_samples_split", 2, 20),
                        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 20),
                        "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2", None]),
                        "bootstrap": trial.suggest_categorical("bootstrap", [True, False]),
                    }

                    model = RandomForestRegressor(**param, random_state=42, n_jobs=-1)
                    model.fit(X_train, y_train)

                    y_pred = model.predict(X_val)
                    mse = mean_squared_error(y_val, y_pred)
                    
                    return mse
                
                study = optuna.create_study(direction="minimize")
                study.optimize(objective, n_trials=50, timeout=3600)



                if epoch_mse < best_mse:
                    best_model = epoch_model
                    best_mse = epoch_mse
                    best_mae = epoch_mae
                    best_device = dev
                
                # aggregate test data
                sum_results = epoch_model.predict(X_test) if sum_results is None else sum_results + epoch_model.predict(X_test)
            
            
            total_mse += best_mse
            total_mae += best_mae
            
            print("current problem: {}/200 | Best MSE: {} | Best MAE: {} in Device {}".format(idx+1, best_mse, best_mae, best_device))
            
            # epoch_result = np.round(sum_model.predict(X_test), 2)
            epoch_result = sum_results / model_len
            result.extend(epoch_result)
        # display(X_test)

    print("All questions finished, Total MSE: {} | Total MAE: {}".format(total_mse, total_mae))
    

    return result



result = run()
# with open('result.txt', 'w') as f:
#     for line in result:
#         f.write(f"{line}\n")

    
    

current problem: 1/200 | Best MSE: 23981.65274730238 | Best MAE: 74.90160891756884 in Device 8
current problem: 2/200 | Best MSE: 3033.1222261802263 | Best MAE: 17.89892878542092 in Device 2
current problem: 3/200 | Best MSE: 3033.1222261802263 | Best MAE: 17.89892878542092 in Device 2
current problem: 4/200 | Best MSE: 9019.825194378844 | Best MAE: 38.505882386677186 in Device 14
current problem: 5/200 | Best MSE: 2265.674863611017 | Best MAE: 20.43697482393596 in Device 3
current problem: 6/200 | Best MSE: 2265.674863611017 | Best MAE: 20.43697482393596 in Device 3
current problem: 7/200 | Best MSE: 2265.674863611017 | Best MAE: 20.43697482393596 in Device 3
current problem: 8/200 | Best MSE: 2555.733270143797 | Best MAE: 19.10541910304912 in Device 4
current problem: 9/200 | Best MSE: 1633.4484075095315 | Best MAE: 20.329363159698584 in Device 6
current problem: 10/200 | Best MSE: 1192.85348156771 | Best MAE: 16.247672947376024 in Device 7
current problem: 11/200 | Best MSE: 1192.85

In [13]:
target_dir = os.getcwd() + f"/results/upload.csv"
upload = pd.read_csv(target_dir)

# result = [0 if r<0 else r for r in result]
result = np.round(result, 2)
upload['答案'] = result
upload.to_csv(os.getcwd() + "/results/upload_v3.csv", index=False)

In [12]:

result

array([-11.31, 267.  , 235.01, ...,   0.71,   2.54, -13.72])

In [12]:
d = load_date_device_dict()
data_ls = load_data()

intersection, target, X_test = get_intersection_date(data_ls, 1, 8, '20240117')

X_train, X_val, y_train, y_val = split_train_val_test(intersection, target)
# X_train, X_val, X_test = normalize_features(X_train, X_val, X_test)
X_train

,Date,Month,Day,Hour,WindSpeed,Pressure,Temperature,Humidity,Sunlight,sum_ele,sum_pos,high,ele,Power,Minute
4586,145.0,5,25,12,0.000,1007.617,30.780,93.407,12225.666,0.101524,0.993252,45.99,-0.62932,34.965,40
2251,52.0,2,22,8,1.603,1014.837,22.951,71.445,4416.749,0.001776,0.999630,45.99,-0.62932,4.201,10
4653,147.0,5,27,10,0.000,1000.158,32.341,87.409,16333.750,0.104371,0.990432,45.99,-0.62932,61.104,50
605,19.0,1,20,16,0.000,1010.201,23.849,71.692,918.083,0.000000,0.690738,45.99,-0.62932,0.140,50
1194,30.0,1,31,16,0.000,1014.579,23.980,71.154,3919.582,0.000000,0.118461,45.99,-0.62932,3.004,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3772,123.0,5,3,7,0.000,1010.737,24.283,100.000,6113.249,0.068583,-0.996541,45.99,-0.62932,8.942,20
5191,163.0,6,12,10,0.000,1001.517,32.747,84.956,12540.750,0.132656,-0.440891,45.99,-0.62932,35.269,30
5226,163.0,6,12,16,0.000,999.062,33.218,83.757,7768.084,0.133098,-0.412821,45.99,-0.62932,14.378,20
5390,168.0,6,17,7,0.000,1002.968,31.436,99.850,5725.002,0.141487,0.163507,45.99,-0.62932,7.586,10


In [13]:
rf_model = RandomForestRegressor(n_estimators=100, max_depth=6, random_state=42, verbose=0)
# rf_model = XGBRegressor(n_estimators=100, random_state=42)
# rf_model = LGBMRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# 預測測試集的發電量
y_pred = rf_model.predict(X_val)

# 評估模型：計算RMSE
mse = mean_squared_error(y_val, y_pred)
mae = mean_absolute_error(y_val, y_pred)
print(f"MSE: {mse}")
print(f"MAE: {mae}")

# 顯示模型的預測結果
print("預測發電量:", y_pred)
print("實際發電量:", y_val.values)

# 顯示特徵重要性
print("特徵重要性:", rf_model.feature_importances_)

MSE: 55104.74670014146
MAE: 135.44472574710656
預測發電量: [206.81347497 365.06268881  13.08113157 ... 161.85265728  13.08113157
  16.87614783]
實際發電量: [2.63090e+01 1.49484e+02 9.57000e-01 ... 9.43170e+01 9.90000e-02
 1.48340e+01]
特徵重要性: [2.31981285e-02 4.66155893e-04 4.21869990e-03 1.04420830e-01
 4.55950275e-02 2.65572276e-02 2.45711933e-02 1.34604116e-01
 3.60075505e-01 7.31602698e-03 1.60879644e-03 0.00000000e+00
 0.00000000e+00 2.67078884e-01 2.89409923e-04]


In [24]:
sum(y_pred<0)

324

In [98]:
np.set_printoptions(suppress=True)
pred = np.round(rf_model.predict(X_test), 2)

In [99]:
y = np.array([1277.55,1369.8,  1462.67, 1540.4,  1594.3,  1631.14, 1659.6,  1671.31, 1674.38,
 1669.7,  1689.69, 1708.54, 1710.23, 1734.19, 1727.19, 1760.06, 1766.71, 1728.95,
 1705.11, 1703.99, 1733.1,  1664.08, 1791.86, 1791.78, 1329.92,  417.97,  291.34,
  195.89,  160.57,  122.99,  105.78,   81.01,   56.43,   36.42,   26.28,   22.13,
   22.02,   19.7,    15.99,   14.8,    12.8,     9.86,    6.89,    4.8,     3.92,
    3.5,     1.72,    0.58])

mean_absolute_error(pred, y)

148.87354166666665

In [ ]:
# for k, v in output.items():
#         if 0 not in v:
#             print(k)
        
#     with open("xtest_num.txt", 'w') as file:
#         file.write(json.dumps(output))

# 156
# 157
# 183
# 189
# 191
# 192